In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch.optim as optim
import pandas as pd
import os
import time
import pickle as pkl
import copy

In [2]:
model_count = 0

In [3]:
from train_model import train_model

In [4]:
with open('tiny-imagenet-200/words.txt', 'r') as f:
    image_dict = {}
    for line in f:
        line = (line.strip('\n').split('\t'))
        image_dict[line[0]] = line[1].split(',')[0]

In [5]:
with open('tiny-imagenet-200/wnids.txt', 'r') as f:
    image_labels = {}
    for line in f:
        image_labels[line.strip('\n')] = image_dict[line.strip('\n')]

In [6]:
# with open('tiny-imagenet-200/val/val_annotations.txt') as f:
#     val_labels = {}
#     for line in f:
#         print(line)

In [7]:
# Code used from https://github.com/tjmoon0104/pytorch-tiny-imagenet/blob/master/Final_Test.ipynb

%time data_transforms = {\
    'train': transforms.Compose({\
        transforms.ToTensor()}),\
    'test': transforms.Compose({\
        transforms.ToTensor()}),\
    'val': transforms.Compose({\
        transforms.ToTensor()})\
}

data_dir = 'tiny-imagenet-200/'
num_workers = {
    'train' : 100,
    'val' : 0,
    'test' : 0
}

%time image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])\
                                       for x in ['train', 'val', 'test']}
%time dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,\
               shuffle=True, num_workers=num_workers[x]) for x in ['train', 'test', 'val']}

%time dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

CPU times: user 7 µs, sys: 6 µs, total: 13 µs
Wall time: 18.4 µs
CPU times: user 456 ms, sys: 90.1 ms, total: 546 ms
Wall time: 804 ms
CPU times: user 133 µs, sys: 0 ns, total: 133 µs
Wall time: 138 µs
CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.3 µs


In [8]:
image_datasets['val'].classes == image_datasets['train'].classes

True

In [9]:
# device = torch.device('cpu')

In [10]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  
# If 'cuda:0' is printed, it means GPU is available.

cuda:0


In [199]:
# # ### New Model ###
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, padding=1)
#         self.pool1 = nn.AvgPool2d(2, 2)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, padding=1)
        
#         self.fc1   = nn.Linear(30*8*8, 1000)
#         self.fc2   = nn.Linear(1000, 200)
#         #self.fc3   = nn.Linear(2000, 200)
#         #self.fc4   = nn.Linear(900, 200)
        
#         self.dropout = nn.Dropout(p=0.2)

#     def forward(self, x):
#         x = self.pool1(F.leaky_relu(self.conv1(x)))
#         #print(x.size())
#         x = self.pool1(F.leaky_relu(self.conv2(x)))
#         x = self.pool1(F.leaky_relu(self.conv3(x)))
#         #print(x.size())
#         x = self.dropout(x.view(-1, 30*8*8))
#         #print(x.size())
        
#         x = F.leaky_relu(self.fc1(x))
#         #print(x.size())
#         x = F.leaky_relu(self.fc2(x))
#         return x

# net = Net()     # Create the network instance.
# net.to(device)  # Move the network parameters to the specified device.

Net(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1920, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [226]:
### Initial Model ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, padding=1)
        self.pool  = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1)
        self.fc1   = nn.Linear(20*16*16, 2000)
        self.fc2   = nn.Linear(2000, 200)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.size())
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x = x.view(-1, 20*16*16)
        #print(x.size())

        x = F.relu(self.fc1(x))
        #print(x.size())

        x = self.fc2(x)
        #print(x.size())

        return x

net = Net()     # Create the network instance.
net.to(device)  # Move the network parameters to the specified device.

Net(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=5120, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=200, bias=True)
)

In [227]:
# The function to show an image.
def imshow(img):
    img = img / 2 + 0.5     # Unnormalize.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# # Get some random training images.
# dataiter = iter(dataloaders['train'])
# images, labels = dataiter.next()
# # Show images.
# imshow(torchvision.utils.make_grid(images))
# # Print labels.
# print(' '.join('%5s' % classes[labels[j]] for j in range(100 )))

In [228]:
classes = [image_dict[image_datasets['train'].classes[i]] for i in range(len(image_datasets['val'].classes))]

In [229]:
l2_lambda = 0.0001
l1_lambda = 0

In [230]:
## Loading ResNet
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
# move the input and model to GPU for speed if available
model.fc = nn.Linear(512, len(classes))
if torch.cuda.is_available():
    model.to('cuda')

Using cache found in /tmp/xdg-cache/torch/hub/pytorch_vision_v0.6.0


In [231]:
### Loading Custom Net
model = Net()
# model.load_state_dict(torch.load('newmodel_dict.pth'))
# model.eval()
model.to('cuda')

Net(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=5120, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=200, bias=True)
)

In [232]:
# We use cross-entropy as loss function.
loss_func = nn.CrossEntropyLoss()
# We use stochastic gradient descent (SGD) as optimizer.
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=l2_lambda) 

In [233]:
(model.state_dict())['conv1.weight'][0][0]

tensor([[ 0.0520,  0.0197,  0.0643],
        [-0.1458, -0.0164, -0.0610],
        [-0.0704,  0.1244,  0.1471]], device='cuda:0')

In [234]:
type(model)

__main__.Net

In [235]:
def train_model(model, dataloaders, criterion, optimizer, output_file=open('default.txt', 'w+'), l1_lambda=0, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1), file=output_file)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10, file=output_file)
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        l1 = 0
                        if l1_lambda != 0:
                            for param in model.parameters():
                                l1 += param.abs().sum()
                            loss += (l1_lambda * l1)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), file=output_file)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print('',file=output_file)
        print('')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), file=output_file)
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc), file=output_file)
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    #print(val_acc_history)
    return model, val_acc_history

In [236]:
output_fname = 'train_val_outputs/initmodel_l1-{}_l2-{}_outputs.txt'.format(l1_lambda, l2_lambda)
output_fname

'train_val_outputs/newmodel_l1-0_l2-0.0001_outputs.txt'

In [237]:
model

Net(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=5120, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=200, bias=True)
)

In [238]:
%time train_model(model, dataloaders, loss_func, opt, num_epochs=50, output_file=open(output_fname, 'w+'), l1_lambda=l1_lambda)

Epoch 0/49
----------
train Loss: 5.2963 Acc: 0.0059
val Loss: 5.2915 Acc: 0.0065

Epoch 1/49
----------
train Loss: 5.2715 Acc: 0.0085
val Loss: 5.2110 Acc: 0.0179

Epoch 2/49
----------
train Loss: 5.0465 Acc: 0.0260
val Loss: 4.9386 Acc: 0.0375

Epoch 3/49
----------
train Loss: 4.8655 Acc: 0.0472
val Loss: 4.8203 Acc: 0.0545

Epoch 4/49
----------
train Loss: 4.7252 Acc: 0.0636
val Loss: 4.6758 Acc: 0.0690

Epoch 5/49
----------
train Loss: 4.5967 Acc: 0.0788
val Loss: 4.5934 Acc: 0.0776

Epoch 6/49
----------
train Loss: 4.4998 Acc: 0.0902
val Loss: 4.5274 Acc: 0.0861

Epoch 7/49
----------
train Loss: 4.4090 Acc: 0.1005
val Loss: 4.4560 Acc: 0.0965

Epoch 8/49
----------
train Loss: 4.3165 Acc: 0.1129
val Loss: 4.4056 Acc: 0.0985

Epoch 9/49
----------
train Loss: 4.2301 Acc: 0.1227
val Loss: 4.3593 Acc: 0.1072

Epoch 10/49
----------
train Loss: 4.1491 Acc: 0.1351
val Loss: 4.3398 Acc: 0.1076

Epoch 11/49
----------
train Loss: 4.0682 Acc: 0.1457
val Loss: 4.2978 Acc: 0.1136

Ep

(Net(
   (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
   (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (fc1): Linear(in_features=5120, out_features=2000, bias=True)
   (fc2): Linear(in_features=2000, out_features=200, bias=True)
 ),
 [tensor(0.0065, device='cuda:0', dtype=torch.float64),
  tensor(0.0179, device='cuda:0', dtype=torch.float64),
  tensor(0.0375, device='cuda:0', dtype=torch.float64),
  tensor(0.0545, device='cuda:0', dtype=torch.float64),
  tensor(0.0690, device='cuda:0', dtype=torch.float64),
  tensor(0.0776, device='cuda:0', dtype=torch.float64),
  tensor(0.0861, device='cuda:0', dtype=torch.float64),
  tensor(0.0965, device='cuda:0', dtype=torch.float64),
  tensor(0.0985, device='cuda:0', dtype=torch.float64),
  tensor(0.1072, device='cuda:0', dtype=torch.float64),
  tensor(0.1076, device='cuda:0', dtype=torch.float64),
  tensor(0.1136, device='cuda:0',

In [239]:
torch.save(model.state_dict(),'models/l1-{}_l2-{}_initmodel_modelstate.pth'.format(l1_lambda, l2_lambda))

In [31]:
# Check several images.
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(100)))
outputs = net(images.to(device))
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(100)))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    w.join()
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    assert self._parent_pid == os.getpid(), 'can only join a child process'
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
AssertionError: can

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can o

    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter._

NameError: name 'imshow' is not defined

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2b2f4cd840>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.7/multipr

In [ ]:
%%time
# Get test accuracy.
correct = 0
total = 0
with torch.no_grad():
    for data in dataloaders['train']:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(correct/total)

print('Accuracy of the network on the 10000 validation images: %d %%' % (
    100 * correct / total))

4
8
12
16
19
22
26
30
34
38
42
46
50
54
58
62
66
70
74
77
81
85
89
92
96
100
103
107
111
115
118
121
125
129
131
135
138
142
145
149
153
156
160
164
167
171
175
179
183
186
190
194
197
201
205
207
211
215
219
223
227
231
235
238
242
246
249
252
256
260
263
266
270
274
278
282
286
290
293
297
300
304
308
312
315
319
322
326
329
333
337
341
345
348
352
356
359
363
366
370
373
377
381
383
387
391
394
398
402
406
410
414
418
422
426
430
434
438
442
444
448
452
456
460
464
468
472
476
480
484
488
490
494
497
501
505
509
513
515
519
523
527
531
535
539
543
547
551
555
559
563
567
571
575
579
583
586
590
594
598
601
605
609
613
617
621
625
629
633
636
639
643
647
651
655
659
663
667
671
674
678
682
686
689
693
695
698
702
706
710
714
717
721
725
728
731
734
738
742
746
750
754
758
761
765
769
773
777
781
783
786
789
793
797
801
805
809
813
817
821
825
829
833
837
841
845
849
853
857
860
864
868
872
876
879
881
885
888
892
895
898
902
906
910
914
918
922
926
930
934
938
941
945
948
952
956
960

6368
6371
6375
6379
6383
6387
6391
6395
6399
6402
6406
6409
6413
6416
6420
6424
6428
6432
6436
6440
6444
6448
6452
6456
6460
6464
6468
6472
6476
6480
6483
6486
6490
6494
6498
6502
6506
6509
6513
6516
6520
6524
6528
6531
6535
6539
6543
6547
6551
6554
6558
6562
6566
6568
6572
6576
6580
6583
6587
6590
6594
6596
6600
6604
6608
6612
6616
6619
6623
6626
6630
6633
6636
6640
6642
6646
6650
6654
6658
6662
6665
6669
6672
6676
6680
6683
6687
6691
6695
6698
6702
6706
6710
6714
6718
6722
6726
6730
6734
6737
6741
6745
6749
6753
6757
6761
6765
6769
6772
6776
6780
6783
6787
6791
6795
6799
6803
6807
6811
6815
6819
6823
6826
6830
6833
6837
6841
6845
6849
6853
6856
6858
6861
6865
6868
6871
6874
6878
6882
6884
6887
6891
6895
6897
6901
6905
6909
6913
6917
6921
6925
6928
6932
6936
6940
6944
6948
6952
6956
6960
6964
6968
6972
6976
6979
6983
6986
6990
6994
6998
7001
7005
7009
7013
7015
7018
7022
7026
7030
7034
7038
7042
7046
7050
7052
7056
7059
7063
7067
7071
7073
7077
7080
7083
7087
7091
7095
7099
7103
7106


12079
12083
12087
12090
12094
12098
12102
12106
12109
12112
12116
12120
12124
12128
12132
12136
12140
12144
12148
12152
12155
12159
12163
12167
12171
12175
12179
12183
12187
12191
12195
12199
12203
12207
12211
12215
12219
12222
12226
12230
12234
12238
12242
12246
12250
12253
12257
12260
12264
12268
12272
12276
12279
12283
12286
12290
12293
12297
12300
12302
12305
12308
12312
12314
12317
12321
12325
12329
12333
12337
12341
12345
12348
12351
12355
12359
12363
12367
12369
12373
12377
12380
12384
12388
12392
12395
12398
12402
12406
12410
12413
12417
12420
12424
12428
12431
12435
12439
12443
12447
12451
12455
12459
12463
12466
12470
12472
12475
12479
12482
12486
12490
12494
12498
12502
12506
12510
12514
12518
12522
12526
12530
12534
12538
12542
12546
12550
12553
12556
12560
12564
12568
12572
12575
12579
12583
12587
12591
12595
12598
12602
12605
12608
12611
12615
12618
12622
12626
12629
12633
12637
12641
12645
12649
12652
12656
12660
12663
12667
12671
12674
12677
12681
12684
12688
12692
1269

17150
17153
17157
17161
17165
17169
17173
17176
17180
17184
17187
17191
17195
17199
17203
17207
17211
17215
17219
17223
17226
17230
17234
17238
17242
17246
17250
17254
17258
17262
17266
17270
17274
17278
17282
17286
17288
17291
17295
17299
17302
17306
17310
17313
17316
17320
17324
17328
17332
17335
17339
17343
17347
17351
17355
17359
17363
17367
17370
17373
17377
17381
17385
17387
17390
17394
17397
17401
17404
17408
17412
17416
17418
17422
17425
17429
17432
17436
17440
17444
17448
17452
17456
17458
17461
17465
17468
17472
17476
17478
17482
17486
17489
17492
17496
17500
17503
17507
17511
17514
17518
17522
17525
17529
17532
17536
17539
17541
17545
17549
17553
17555
17559
17562
17566
17570
17573
17577
17581
17585
17589
17593
17596
17600
17604
17608
17612
17616
17620
17623
17626
17630
17634
17638
17642
17645
17649
17653
17656
17659
17662
17665
17668
17672
17676
17680
17684
17688
17691
17695
17699
17702
17705
17709
17713
17717
17721
17724
17728
17732
17736
17740
17744
17748
17752
17754
1775

22250
22254
22258
22262
22265
22269
22273
22277
22281
22285
22289
22292
22296
22300
22303
22307
22310
22314
22318
22322
22326
22329
22333
22337
22340
22343
22347
22351
22354
22358
22362
22366
22370
22374
22378
22382
22386
22389
22393
22397
22401
22405
22409
22413
22415
22418
22421
22425
22429
22433
22437
22441
22445
22449
22453
22457
22460
22464
22468
22472
22476
22480
22484
22488
22492
22494
22498
22502
22506
22508
22511
22515
22518
22522
22526
22530
22533
22537
22541
22545
22549
22553
22557
22560
22564
22568
22572
22576
22580
22584
22587
22591
22595
22599
22602
22606
22610
22613
22617
22621
22624
22627
22631
22635
22639
22642
22646
22650
22654
22657
22661
22665
22669
22673
22676
22679
22682
22685
22688
22692
22696
22700
22704
22708
22712
22715
22718
22722
22726
22730
22734
22738
22742
22746
22750
22754
22757
22761
22764
22768
22772
22775
22779
22783
22787
22791
22795
22799
22802
22806
22810
22814
22818
22821
22824
22828
22831
22835
22838
22842
22846
22849
22853
22857
22860
22864
2286

27349
27353
27357
27360
27364
27368
27372
27375
27378
27382
27386
27390
27394
27398
27402
27405
27408
27412
27416
27420
27424
27428
27432
27436
27440
27442
27446
27450
27454
27458
27461
27464
27468
27472
27476
27479
27481
27485
27489
27493
27497
27499
27503
27507
27511
27515
27519
27523
27527
27531
27535
27539
27543
27547
27551
27555
27557
27560
27563
27566
27570
27574
27578
27581
27585
27588
27592
27595
27599
27603
27607
27610
27613
27616
27619
27623
27627
27630
27633
27637
27641
27645
27648
27650
27654
27658
27661
27665
27669
27672
27676
27680
27684
27688
27692
27696
27700
27704
27707
27711
27715
27719
27723
27727
27731
27735
27739
27742
27745
27749
27753
27757
27761
27764
27768
27772
27776
27780
27784
27788
27792
27795
27799
27803
27807
27811
27815
27818
27822
27826
27830
27832
27836
27840
27844
27848
27852
27855
27859
27863
27866
27869
27873
27877
27881
27885
27888
27892
27895
27899
27903
27907
27910
27914
27918
27922
27926
27929
27932
27936
27940
27944
27948
27951
27955
27959
2796

In [140]:
%%time
# Get test accuracy for each class.
class_correct = list(0. for i in range(200))
class_total = list(0. for i in range(200))
with torch.no_grad():
    for data in dataloaders['val']:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of goldfish :  0 %
Accuracy of European fire salamander :  0 %
Accuracy of bullfrog :  0 %
Accuracy of tailed frog :  0 %
Accuracy of American alligator :  0 %
Accuracy of boa constrictor :  0 %
Accuracy of trilobite :  0 %
Accuracy of scorpion :  0 %
Accuracy of black widow :  0 %
Accuracy of tarantula :  0 %
